Importamos o requests e o bs4 para fazer um requisição HTTP "get" do site e depois fazemos o scrapping do site pegando o conteúdo de interesse

In [49]:
import requests
from bs4 import BeautifulSoup
import re

In [71]:
def get_poems_authors(url):
    r = requests.get(url)
    print(f"The url: {url} status code it's {r.status_code}")

    html_doc = r.text
    soup = BeautifulSoup(html_doc)

    authors = set()
    poems = []

    for span in soup.find_all("span", class_="autor"):
        author = span.text
        authors.add(author)

    for span in soup.find_all("p", class_=re.compile(r"frase\s.*")):
        poem = span.text
        poems.append(poem)

    current_page = soup.find("span", class_="atual")
    next_page = current_page.find_next("a")
    
    href_next_page = next_page['href']

    if 'poemas' not in href_next_page:
        href_next_page = False

    return href_next_page,authors,poems

Em razão da paginação do site, temos que buscar qual seria a próxima página, retornando o href como parâmetro na função acima e fazendo o scrpping da nova página

In [ ]:
main_page = "https://www.pensador.com"
poem_page = "https://www.pensador.com/poemas/"
href_exists = True

while href_exists:
    href,authors,poems = get_poems_authors(poem_page)

    href_exists = True if href else False
    
    if not href_exists:
        break

    poem_page = main_page+href